In [ ]:
import pandas as pd
import numpy as np
import io
import re
import math

In [ ]:
# ganti nama file hasil proses sentiwordnet yang didapat dari tahap sebelumnya
file = pd.read_csv('data_DIC_INDOSATCARE.csv', delimiter=',')
file.insert(0, "ID", range(0,len(file)))
file.set_index("ID")

,tweet_id,term,pos,pos_fre,neg_fre,label
ID,,,,,,
0,0,sampah,n,3,1,0
1,2,cek,n,2,0,0
2,2,tolong,v,5,0,0
3,3,hilang,v,0,10,0
4,4,malam,n,0,1,1
...,...,...,...,...,...,...
1655,992,kasih,n,4,0,0
1656,992,bintang,n,1,0,0
1657,992,jijik,adj,0,3,0


In [ ]:
file.isnull().sum()

ID          0
tweet_id    0
term        0
pos         0
pos_fre     0
neg_fre     0
label       0
dtype: int64

In [ ]:
# menambahkan 1 pada setiap frekuensi kata yang didapat untuk menangani jumlah frekuensi = 0
file['pos_fre'] += 1
file['neg_fre'] += 1

![title](pic2.png)

In [ ]:
N = len(file)
print("nilai N", N)

nilai N 1660


In [ ]:
file['pos_fre'].sum()

4846

In [ ]:
file['neg_fre'].sum()

3663

In [ ]:
feature_weight = []

In [ ]:
# %%time
feature_weight.clear()

# code untuk implementasi rumus PMI
for itter in range(len(file)):
    selected_term = file.loc[itter]
    # print(selected_term['term'])

    others_term = file[file['ID'] != itter]
    # print(selected_term['label'])
    if selected_term['label'] == 0:
        A = selected_term['neg_fre']
        B = selected_term['pos_fre']
        C = others_term['neg_fre'].sum()
        tmp = (A*N) / ((A+B)*(A+C))
        result = math.log2(tmp)
    
    elif selected_term['label'] == 1:
        A = selected_term['pos_fre']
        B = selected_term['neg_fre']
        C = others_term['pos_fre'].sum()
        tmp = (A*N) / ((A+B)*(A+C))
        result = math.log2(tmp)
    
    feature_weight.append(result)
    #print(selected_term['term'],A,B,C,result)

In [ ]:
selected_term

ID          1659
tweet_id     992
term         kek
pos            n
pos_fre        1
neg_fre        2
label          0
Name: 1659, dtype: object

In [ ]:
len(feature_weight)

1660

In [ ]:
file['fea_weight'] = feature_weight

In [ ]:
minA = file['fea_weight'].min()
maxA = file['fea_weight'].max()

newMin = -1
newMax = 1

normalize = []

print(minA, maxA)

-5.867539258104421 -1.215843040918049


In [ ]:
normalize.clear()

for i in range(len(file)):
    selected = file.loc[i]
    v = selected['fea_weight']
    
    # code untuk normalisasi pada bobot yang sudah didapat
    nor_minmax = (((v - minA) / (maxA - minA)) * (newMax - newMin )) + newMin
    
    #print (nor_minmax)
    normalize.append(nor_minmax)  

In [ ]:
len(normalize)

1660

In [ ]:
print(normalize)

[0.3503608796744251, 0.17191522032744233, -0.17520758578991025, 0.9778445116644996, 0.17676024674787905, 0.17676024674787905, 0.7198027615051104, 0.17676024674787905, 0.17676024674787905, 0.7851565964711396, 0.8934033944316564, 0.3503608796744251, 0.8934033944316564, 0.3503608796744251, 0.9821670917275649, 0.8206112591755976, 0.17676024674787905, 0.6067109371241568, -0.17520758578991025, -0.17520758578991025, 0.17676024674787905, 0.17676024674787905, 0.9821670917275649, 0.2850070447083961, 0.679770308806469, 0.6018659107037201, 0.6067109371241568, 0.17676024674787905, 0.6067109371241568, 0.42826527777717405, 0.17676024674787905, 0.6067109371241568, 0.679770308806469, 0.6067109371241568, 0.5062481356156623, 0.9778445116644996, 0.762598193065394, 0.679770308806469, 0.6067109371241568, 0.17676024674787905, 0.42826527777717405, 0.6067109371241568, 0.17676024674787905, 0.17676024674787905, 0.679770308806469, 0.6067109371241568, 0.679770308806469, 0.17676024674787905, 0.6067109371241568, 0.1

In [ ]:
file['normalize_weight'] = normalize

In [ ]:
# menyimpan hasil normalisasi dari pada file CSV. Ganti nama jika dibutuhkan
file.to_csv('OUT_ALL_INDOSAT.csv', encoding='utf-8')